In [1]:
GIANG_OI = "/kaggle/input/ess-vlsp2023-trainset-denoised/VLSP-2023-NEU/audios"
TTH = "/kaggle/input/tts-tth-lmh/TTH_final/wav"

# output path
__OUTPATH__ = "."

In [2]:
# ALL AUDIO IN 22050 HZ
SPK_DIRS = [GIANG_OI, TTH]
SPK_NAMES = [0, 1]

In [3]:
import os
from scipy.io import wavfile
from pydub import AudioSegment

from pydub import AudioSegment
from pydub.silence import split_on_silence
import os

def split(sound):
    dBFS = sound.dBFS
    chunks = split_on_silence(sound,
        min_silence_len = 100,
        silence_thresh = dBFS-16,
        keep_silence = 100
    )
    return chunks

def combine(_src):
    audio = AudioSegment.empty()
    for i,filename in enumerate(os.listdir(_src)):
        if filename.endswith('.wav'):
            filename = os.path.join(_src, filename)
            audio += AudioSegment.from_wav(filename)
    return audio

def save_chunks(chunks, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    counter = 0

    target_length = 5 * 1000
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < target_length:
            output_chunks[-1] += chunk
        else:
            # if the last output chunk is longer than the target length,
            # we can start a new one
            output_chunks.append(chunk)

    for chunk in output_chunks:
        chunk = chunk.set_frame_rate(24000)
        chunk = chunk.set_channels(1)
        counter = counter + 1
        chunk.export(os.path.join(directory, str(counter) + '.wav'), format="wav")

In [29]:
import shutil

for spk_dir, spk_name in zip(SPK_DIRS, SPK_NAMES):
    print("Processing speaker: ", spk_name)
    for i, filename in enumerate(os.listdir(spk_dir)):
        if filename.endswith('.wav'):
            filename = os.path.join(spk_dir, filename)
            sound = AudioSegment.from_wav(filename)
            chunks = split(sound)
            save_chunks(chunks, os.path.join(__OUTPATH__, spk_name, str(i)))

Processing speaker:  tth


In [8]:
data_list = []
for path, subdirs, files in os.walk(__OUTPATH__):
    for name in files:
        if name.endswith(".wav"):
            speaker = path.split('/')[-2].replace('p', '')
            data_list.append([os.path.join(path, name), speaker])

import pandas as pd

data_list = pd.DataFrame(data_list)
data_list = data_list.sample(frac=1)

import random

split_idx = round(len(data_list) * 0.1)

test_data = data_list[:split_idx]
train_data = data_list[split_idx:]

data_list.head()

,0,1
10686,./1/5290/1.wav,1
9659,./1/4263/1.wav,1
3674,./0/3138/1.wav,0
9294,./1/3898/1.wav,1
14507,./1/9111/1.wav,1


In [9]:
train_data.columns = ['Path', 'Speaker']
test_data.columns = ['Path', 'Speaker']

In [10]:
train_data["Path"] = train_data["Path"].apply(lambda x: x.replace('./', './Data/VLSP/'))
test_data["Path"] = test_data["Path"].apply(lambda x: x.replace('./', './Data/VLSP/'))

/tmp/ipykernel_236741/623454426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Path"] = train_data["Path"].apply(lambda x: x.replace('./', './Data/VLSP/'))
/tmp/ipykernel_236741/623454426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["Path"] = test_data["Path"].apply(lambda x: x.replace('./', './Data/VLSP/'))


In [11]:
# write to file 

file_str = ""
for index, k in train_data.iterrows():
    file_str += k['Path'] + "|" +str(k['Speaker'])+ '\n'
text_file = open(__OUTPATH__ + "/train_list.txt", "w")
text_file.write(file_str)
text_file.close()

file_str = ""
for index, k in test_data.iterrows():
    file_str += k['Path'] + "|" + str(k['Speaker']) + '\n'
text_file = open(__OUTPATH__ + "/val_list.txt", "w")
text_file.write(file_str)
text_file.close()